In [176]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import re 
import datetime

%matplotlib inline

# TODO 

* Only use complete days, eliminate first day possibly

In [177]:
df = pd.read_csv('./commute_info.csv')

#manipulations
df['time'] = pd.to_datetime(df['time'])
#Making EST
df['time'] = df['time'] - pd.Timedelta(4, unit='h')
df['time'] = df.time.dt.round('1min')
df['date'] = df.time.dt.date
df = df.drop(df.loc[df['date'] == datetime.date(2018, 10, 31)].index).reset_index(drop=True)

In [178]:
def duration_clean(mins):
    if 'h' in mins:
        vals = re.findall(r'\d+', mins)
        return int(vals[0]) * 60 + int(vals[1])
    else:
        return int(re.search(r'\d+', mins).group())
    
df['trip_length'] = df['duration_in_traffic'].apply(duration_clean)

In [179]:
def morning(time):
    if time.hour < 11:
        return 1
    else:
        return 0
    
df['is_morning'] = df.time.apply(morning)
df['hour'] = df.time.dt.time
df['is_weekday'] = df.time.apply(lambda x: 1 if x.dayofweek < 5 else 0)

In [180]:
morning = df.loc[df['is_morning'] == 1].copy()
evening = df.loc[df['is_morning'] == 0].copy()

In [181]:
dates = list(morning['date'].unique())
traces = []

for date in dates:
    morn = morning.loc[morning['date'] == date].copy()
    color = 'blue'
    if morn['is_weekday'].max() == 0:
        color = 'red'
    trace0 = go.Scatter(x=morn['hour'], 
                        y=morn['trip_length'], 
                        name=str(date), 
                        showlegend=False,
                        line=dict(color=color))
    traces.append(trace0)
    
data = traces

layout = go.Layout(xaxis=dict(title='Time (EST)'), 
                   yaxis=dict(title='Duration of Trip (Minutes)',
                              range=[0, 70]))

fig = go.Figure(data=data, layout=layout)

py.offline.plot(fig, filename='commute_times.html')

'file:///Users/slevin886/Desktop/DataProjects/trip_duration/commute_times.html'

In [183]:
dates = list(evening['date'].unique())
traces = []

for date in dates:
    morn = evening.loc[evening['date'] == date].copy()
    color = 'blue'
    if morn['is_weekday'].max() == 0:
        color = 'red'
    trace0 = go.Scatter(x=morn['hour'], 
                        y=morn['trip_length'], 
                        name=str(date), 
                        showlegend=False,
                        line=dict(color=color))
    traces.append(trace0)
    
data = traces

layout = go.Layout(xaxis=dict(title='Time (EST)'), 
                   yaxis=dict(title='Duration of Trip (Minutes)',
                              range=[0, 70]))

fig = go.Figure(data=data, layout=layout)

py.offline.plot(fig, filename='commute_times.html')

'file:///Users/slevin886/Desktop/DataProjects/trip_duration/commute_times.html'